In [2]:
%matplotlib inline
import matplotlib
import matplotlib.patches as mpatches
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle
from io import BytesIO
import random
import itertools as it

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder

from kaggle_camera_model_id_lib.utils import PechkaBot, ImageList, NpzFolder, NCrops, TifFolder, TifFolderExFiles
from kaggle_camera_model_id_lib.models import VggHead, StyleVggHead, IEEEfcn, ResNetFC, ResNetX, FatNet1
from kaggle_camera_model_id_lib.models import InceptionResNetV2fc, InceptionResNetV2fcSmall
from kaggle_camera_model_id_lib.utils import jpg_compress, equalize_v_hist, hsv_convert
from kaggle_camera_model_id_lib.utils import scale_crop_pad, gamma_correction
from kaggle_camera_model_id_lib.utils import patch_quality_dich, n_random_crops, n_pseudorandom_crops
from kaggle_camera_model_id_lib.models import DANet, ResNetFeatureExtractor, AvgFcClassifier, FCDiscriminator
from kaggle_camera_model_id_lib.models import AvgClassifier

from sklearn.manifold import TSNE

from scipy.spatial.distance import pdist, squareform

import imgaug as ia
from imgaug import augmenters as iaa

In [4]:
scale_05 = lambda img: scale_crop_pad(img, 0.5)
scale_08 = lambda img: scale_crop_pad(img, 0.8)
scale_15 = lambda img: scale_crop_pad(img, 1.5)
scale_20 = lambda img: scale_crop_pad(img, 2.0)
gamma_08 = lambda img: gamma_correction(img, 0.8)
gamma_12 = lambda img: gamma_correction(img, 1.2)
jpg_70 = lambda img: jpg_compress(img, (70, 71))
jpg_90 = lambda img: jpg_compress(img, (90, 91))
augs = [scale_05, scale_08, scale_15, scale_20, gamma_08, gamma_12, jpg_70, jpg_90]

def random_aug_kaggle(img, p=0.5):
    if np.random.rand() < p:
        return random.choice(augs)(img)
    return img

In [5]:
def loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')
        
center_crop = transforms.CenterCrop(1520)

train_files = glob('/home/mephistopheies/storage2/data/camera-model-id/raw/train/*/*.*')
print(len(train_files))

output_dir = '/home/mephistopheies/storage2/data/camera-model-id/svm_data/train_img/'

2750


In [17]:
for fname in tqdm_notebook(train_files):
    c = fname.split('/')[-2]
    img = np.array(center_crop(loader(fname)))

    if not os.path.isdir(os.path.join(output_dir, c)):
        os.makedirs(os.path.join(output_dir, c))
        
    fname = os.path.basename(fname)
    
    for ix, crop in enumerate(NCrops(img, crop_size=512, step=500)):
        np.savez(os.path.join(output_dir, c, '%s__%i.npz' % (fname, ix)), 
                 data=crop)
        np.savez(os.path.join(output_dir, c, '%s__aug__%i.npz' % (fname, ix)), 
                 data=random_aug_kaggle(Image.fromarray(crop), p=1.0))